In [2]:
#Omid55
import requests
import pandas as pd
import pickle as pk
import csv
import time

In [3]:
URL = {
    'BASE': 'https://{proxy}.api.pvp.net/api/lol/{region}/v2.2/{rest}',
    'STATIC_BASE': 'https://global.api.pvp.net/api/lol/static-data/{region}/v1.2/{rest}',
    'MATCHLIST_URL': 'matchlist/by-summoner/{summonerId}',
    'MATCH_URL': 'match/{matchId}',
    'CHAMPION_URL': 'champion/{id}?champData=all'
}

REGIONS = {
    'north america': 'na',
    'europe west': 'euw'
}

In [4]:
class Match(object):
    
    def __init__(self):
        self.winners = []
        self.losers = []
        self.duration = -1
        

In [5]:
class RiotAPI(object):
    
    t1 = 0
    
    def __init__(self, api_key, region=REGIONS['europe west']):
        self.api_key = api_key
        self.region = region
        self.champions = {}
        
    def _request(self, base, rest, params={}):
        args = {'api_key': self.api_key}
        args.update(params)
        response = requests.get(
            URL[base].format(
                rest=rest,
                proxy=self.region,
                region=self.region,
            ),
            params=args
        )
        time.sleep(1.2)
        return response.json()
    
    def _base_request(self, rest, params={}):
        return self._request('BASE', rest, params)
        
    def _static_request(self, rest, params={}):
        return self._request('STATIC_BASE', rest, params)
    
    # my functions
    def _get_list_of_match_ids(self, sid):
        rest = URL['MATCHLIST_URL'].format(
            summonerId=sid
        )
        result = self._base_request(rest)
        if 'matches' in result:
            for match in result['matches']:
                yield match['matchId']
                
    def _get_match(self, mid):
        rest = URL['MATCH_URL'].format(
            matchId=mid
        )
        result = self._base_request(rest)
        return result
    
    def get_champion_capabilities(self, championId):
        if championId in self.champions:
            return self.champions[championId]
        else:
            rest = URL['CHAMPION_URL'].format(
                id=championId
            )
            result = self._static_request(rest)
            
            features = []
            for k in sorted(result['info'].keys()):
                features.append(result['info'][k])
            for k in sorted(result['stats'].keys()):
                features.append(result['stats'][k])
            self.champions[championId] = features
            return features
    
    
    def get_matches(self):
        sid = 5
        matchids = self._get_list_of_match_ids(sid)
        for matchid in matchids:
            match = self._get_match(matchid)
            if 'participants' in match:
                #m = Match()
                losers = []
                winners = []
                duration = match['matchDuration']
                for member in match['participants']:
                    if member['stats']['winner']:
                        winners += (self.get_champion_capabilities(member['championId']))
                    else:
                        losers += (self.get_champion_capabilities(member['championId']))
                #yield m
                yield winners + losers + [duration]
            
KEY = 'RGAPI-839ABFA9-5540-49E5-A8B7-AA086DA77F6E'
api = RiotAPI(KEY)

In [ ]:
result = 

In [6]:
header = ''
for i in range(1,6):
    for s in sorted(result['info'].keys()) + sorted(result['stats'].keys()):
        header += s + '_winner' + str(i) + ','
for i in range(1,6):
    for s in sorted(result['info'].keys()) + sorted(result['stats'].keys()):
        header += s + '_loser' + str(i) + ','
header += 'duration'
print(header)

NameError: name 'result' is not defined